# Using btrack with napari


In [1]:
import os
import json

import btrack
import napari

import numpy as np
import pandas as pd

from skimage import io
from napari.utils import nbscreenshot

In [2]:
print(btrack.__version__)

0.4.0


In [3]:
objects = btrack.dataio.import_CSV('napari_example.csv')

In [4]:
objects[0]

,ID,x,y,z,t,dummy,states,label,prob
0,0,517.559509,9.081633,0.0,0,False,0,0,0.0


In [5]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('../models/cell_config.json')

    # append the objects to be tracked
    tracker.append(objects)

    # set the volume (Z axis volume is set very large for 2D data)
    tracker.volume=((0,1600), (0,1200), (-1e5,1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

#     tracker.export('./test3.hdf5', obj_type='obj_type_1')

    # get the tracks in a format for napari visualization
    data, properties, graph = tracker.to_napari(ndim=2)

[INFO][2020/11/04 05:50:28 PM] Loaded btrack: /home/quantumjot/Dropbox/Code/py3/BayesianTracker/btrack/libs/libtracker.so
[INFO][2020/11/04 05:50:28 PM] btrack (v0.4.0) library imported
[INFO][2020/11/04 05:50:28 PM] Setting max XYZ search radius to: 100
[INFO][2020/11/04 05:50:28 PM] Starting BayesianTracker session
[INFO][2020/11/04 05:50:28 PM] Loading configuration file: ../models/cell_config.json
[INFO][2020/11/04 05:50:28 PM] Loading motion model: b'cell_motion'
[INFO][2020/11/04 05:50:28 PM] Objects are of type: <class 'list'>
[INFO][2020/11/04 05:50:29 PM] Set volume to ((0, 1600), (0, 1200), (-100000.0, 100000.0))
[INFO][2020/11/04 05:50:29 PM] Starting tracking... 
[INFO][2020/11/04 05:50:29 PM] Tracking objects in frames 0 to 99 (of 500)...
[INFO][2020/11/04 05:50:29 PM]  - Timing (Bayesian updates: 1.13ms, Linking: 0.21ms)
[INFO][2020/11/04 05:50:29 PM]  - Probabilities (Link: 1.00000, Lost: 0.48361)
[INFO][2020/11/04 05:50:29 PM]  - Stats (Active: 78, Lost: 191, Conflicts 

In [6]:
with napari.gui_qt():
    viewer = napari.Viewer()
    viewer.add_tracks(data, properties=properties, graph=graph, name='tracks')